In [130]:
include("../utils.jl")
data = getdata("../data/13")
track = map(x -> split(x, ""), data)
function display(track)
    for i in 1:length(track)
        println(join(track[i]))
    end
    println()
end
function display(track, carts)
    newtrack = deepcopy(track)
    for c in carts
        newtrack[c[1]][c[2]] = c[3]
    end
    display(newtrack)
end

cart_symbols = [">", "<", "^", "v"]
track_dict = Dict(">"=>"-", "<"=>"-", "^"=>"|", "v"=>"|")

function parse_track(track)
    carts = []
    nrow = length(track)
    ncol = length(track[1])
    for r=1:nrow
        for c=2:ncol
            x = track[r][c]
            if x in cart_symbols
                # Row, col, symbol, numturns
                push!(carts, [r, c, x, 0])
                track[r][c] = track_dict[x]
            end
        end
    end
    return track, carts
end

track_start, carts_start = parse_track(track);

In [131]:
track = deepcopy(track_start)
carts = deepcopy(carts_start)


function step_track(track, carts)
    collision = false
    move_dict = Dict(">"=>[0,1], "<"=>[0,-1], "^"=>[-1,0], "v"=>[1,0])
    corners = ["/", "\\"]
    corner_dict = Dict()
    corner_dict[[">", "\\"]] = "v"
    corner_dict[[">", "/"]] = "^"
    corner_dict[["<", "\\"]] = "^"
    corner_dict[["<", "/"]] = "v"
    corner_dict[["^", "\\"]] = "<"
    corner_dict[["^", "/"]] = ">"
    corner_dict[["v", "\\"]] = ">"
    corner_dict[["v", "/"]] = "<"
    
    crossroad_dict = Dict()
    crossroad_dict[[">", 0]] = "^"
    crossroad_dict[[">", 1]] = ">"
    crossroad_dict[[">", 2]] = "v"
    
    crossroad_dict[["<", 0]] = "v"
    crossroad_dict[["<", 1]] = "<"
    crossroad_dict[["<", 2]] = "^"
    
    crossroad_dict[["^", 0]] = "<"
    crossroad_dict[["^", 1]] = "^"
    crossroad_dict[["^", 2]] = ">"
    
    crossroad_dict[["v", 0]] = ">"
    crossroad_dict[["v", 1]] = "v"
    crossroad_dict[["v", 2]] = "<"
    
    # Get move order
    sort!(carts, by=c->(c[1], c[2]))
    
    cart_at_pos = Dict()
    for (i,c) in enumerate(carts)
        cart_at_pos[c[1:2]] = i
    end
    
    carts_to_delete = Int[]
    for (i,c) in enumerate(carts)
        # If this has already crashed, it can't do any more
        if i in carts_to_delete
            continue
        end
        moveto = c[1:2] + move_dict[c[3]]
        dest = track[moveto[1]][moveto[2]]
        # Check for collision and remove carts
        if moveto in keys(cart_at_pos)
            println("Collision at $(moveto[2]-1),$(moveto[1]-1)")
            push!(carts_to_delete, i)
            push!(carts_to_delete, cart_at_pos[moveto])
            collision = true
            continue
        end
        cart_at_pos[moveto] = i
        delete!(cart_at_pos, c[1:2])
        # Corners / intersections
        if dest in corners
            c[3] = corner_dict[ [c[3], dest] ]
        elseif dest=="+"
            c[3] = crossroad_dict[ [c[3], mod(c[4],3)] ]
            c[4] += 1
        end
        c[1:2] = moveto 
    end
    # Remove collided carts by index
    sort!(carts_to_delete)
    if !isempty(carts_to_delete)
        println("Removing carts $carts_to_delete")
        deleteat!(carts, carts_to_delete)
    end
    return collision
end

function check_collisions(carts)
    hascart = Dict()
    for c in carts
        hascart[c[1:2]] = true
    end
end

#display(track, carts)
while true
    collision = step_track(track, carts)
    #display(track, carts)
    if collision break end
end

Collision at 118,112
Removing carts [12, 13]


In [132]:
track = deepcopy(track_start)
carts = deepcopy(carts_start)

while true
    collision = step_track(track, carts)
    if collision
        println(length(carts))
    end
    if length(carts)==1
        println("Last cart left at $(carts[1][2]-1), $(carts[1][1]-1)")
        break
    end
end

Collision at 118,112
Removing carts [12, 13]
15
Collision at 79,97
Removing carts [10, 11]
13
Collision at 100,120
Removing carts [10, 11]
11
Collision at 125,101
Removing carts [9, 10]
9
Collision at 14,97
Removing carts [7, 8]
7
Collision at 22,108
Removing carts [5, 6]
5
Collision at 27,10
Removing carts [1, 2]
3
Collision at 27,46
Removing carts [2, 3]
1
Last cart left at 50, 21
